In [85]:
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from tabulate import tabulate
from datetime import datetime
from pathlib import Path
import numpy as np
from io import StringIO
import re
import pandas as pd
import requests
import json
import time


In [2]:
def get_ticker_information(ticker):
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}

    ticker_information = requests.get(f"https://efts.sec.gov/LATEST/search-index?keysTyped={ticker}", headers=headers)

    response = ticker_information.content.decode()#["hits"]["hits"][0]["_source"]["entity"]
    entity_name = json.loads(response)["hits"]["hits"][0]["_source"]["entity"]
    
    return entity_name

In [378]:
class Consolidated_Schedule_Investments():
    def __init__(self, ticker, entity_name, url):
        self.ticker = ticker
        self.entity_name = entity_name
        self.url = url
    
    def save_File(self, data, filename):

        with pd.ExcelWriter(filename) as writer:
            for sheet_name, df in data:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        # path = Path(filename)
        # df = pd.DataFrame(data)
        # df.to_csv(path, index=False)
    
    def get_table_of_url(self, driver):

        urls = []
        reporting_dates = []

        table = driver.find_element(By.XPATH, '//*[@id="hits"]/table')
        rows = table.find_elements(By.TAG_NAME, 'tr')
        
        for i in range(1, len(rows)):
            cell = driver.find_element(By.XPATH, f'//*[@id="hits"]/table/tbody/tr[{i}]/td[1]/a')
            data_adsh = cell.get_attribute('data-adsh')
            data_adsh = data_adsh.replace('-','')
            data_file_name = cell.get_attribute('data-file-name')
            cell_reporting_date = driver.find_element(By.XPATH, f'//*[@id="hits"]/table/tbody/tr[{i}]/td[3]').text
            unformated_date = datetime.strptime(cell_reporting_date, "%Y-%m-%d")
            reporting_dates.append(cell_reporting_date)

            url = f"https://www.sec.gov/Archives/edgar/data/{self.ticker}/{data_adsh}/{data_file_name}"
            urls.append(url)
        
        return urls, reporting_dates

    
    # Function to extract the target table based on specific heuristics
    def extract_table(self, soup):
        tables = soup.find_all('table')
        candidate_tables = []

        for table in tables:
            # print(repr(table.text))
            # Heuristics to identify the correct table
            # Example: Select tables with more than 2 rows and 2 columns
            rows = table.find_all('tr')
            if len(rows) > 10:
                i = 0
                while True:
                    if rows[i].text != '\n\n\n\n':
                        cols = rows[i].find_all(['td'])
                        break

                    i +=1
                # if rows[5].text != '\n\n\n\n':
                #     cols = rows[5].find_all(['td'])
                # else:
                #     cols = rows[4].find_all(['td'])
                # print(repr(rows[5].text))
                # if len(cols) > 12:
                if "Initial" in rows[1].text:
                    candidate_tables.append(table)
        
        # If multiple tables match, refine selection logic
        if len(candidate_tables) > 1:
            # Example: Further refine based on specific row or column content
            choices = ["Co-Investments", "Primary Private Investment Funds", "Secondary Private Investment Funds"]
            for table in candidate_tables:
                if any(x in str(table) for x in choices):
                    continue
                else:
                    remove_idx = candidate_tables.index(table)
                    candidate_tables.pop(remove_idx)
            return candidate_tables  # Fallback to the first candidate

        return None
    
    def amg_pantheon_fund(self):

        dataframes = []

        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
        driver.implicitly_wait(5)
        driver.maximize_window()
        driver.get(self.url)
        time.sleep(3)

        

        urls_dates = self.get_table_of_url(driver)
        dates = urls_dates[1]
        urls = urls_dates[0]
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}
        
        for j in range(len(urls)):
            while True:
                r = requests.get(urls[j], headers=headers)
                if r.status_code == 200:
                    soup = BeautifulSoup(r.text, 'html.parser')
                    break

            target_table = self.extract_table(soup)
            firts_text = False
            full_text_table = ""

            for table in target_table:
                if target_table:
                    texts = table.text
                    texts = texts.replace('\xa0', '')
                    texts = texts.replace('\u2003', '')
                    texts = texts.strip('\n\t ')
                    texts = texts.replace('\n \n', '\n\n')
                    texts = texts.replace('Initial\nAcquisition', 'Security\tInitial Acquisition')
                    texts = texts.replace('Initial Acquisition\nDate', 'Security\tInitial Acquisition Date')
                    texts = texts.replace('InitialAcquisitionDate', 'Security\tInitial Acquisition Date')
                    texts = texts.replace('PercentofNet\nAssets', 'Percent of Net Assets')
                    texts = re.sub(r'(?<=\w)(\n)+Secondary', ' \nSecondary',texts)
                    texts = texts.replace(' (continued)', '')
                    texts = texts.replace('\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', '\t\t\t\t\n')
                    texts = texts.replace('\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', '\n')
                    texts = re.sub(r'(?<=%)\n+(?=\$)', '\t', texts)
                    texts = texts.replace('\n\n\n\n\n\n\n\n\n\n\n\n', '\t\t')
                    texts = texts.replace('\n\n\n\n\n\n\n\n\n\n\n\n\n', '\t\t\t\t\n')
                    texts = texts.replace('\n\n\n\n\n\n\n\n\n\n\n', '\t\t')
                    texts = texts.replace('\n\n\n\n\n\n\n\n\n\n', '\t\t\t\t\n')
                    texts = re.sub(r'(?<=[a-zA-Z])\n+(?=[0-9])', '\t', texts)
                    texts = re.sub(r'Net Assets\n+', 'Net Assets\t\t', texts)
                    texts = re.sub(r'Other Assets, less Liabilities\n+', 'Other Assets, less Liabilities\t\t', texts)
                    texts = texts.replace('\n\n\n\n\n\n\n\n\n', ' \n')
                    texts = texts.replace('\n\n\n\n\n\n\n\n', '\n')
                    texts = re.sub(r'\n+(?=\d)', '\t', texts)
                    texts = re.sub(r'(?<=\S)\s*\n{7}(?=\s*\d)', '\t', texts)
                    texts = texts.replace('\n\n\n\n\n\n\n', '\n')
                    texts = texts.replace('\n\n\n\n\n\n', '\n')
                    texts = texts.replace('\n\n\n\n', '\t')
                    texts = texts.replace("\n\n\n Net Assets", "\n Net Assets")
                    texts = texts.replace("\n\n Net Assets", "\n Net Assets")
                    texts = texts.replace('\n\n\n', '\t')
                    texts = texts.replace('\n\nSe', '\n')
                    texts = texts.replace('\n\n', '\t')
                    texts = re.sub(r'\n%', '%', texts)
                    texts = re.sub(r'(\S)\n(\S)', r'\1 \2', texts)
                    texts = re.sub(r'(\S)\n(\s*\()', r'\1 \2', texts)
                    texts = re.sub(r'(?<=\S)\n(?=\d)|(?<=\d)\n(?=\S)', '', texts)
                    texts = re.sub(r'^[ \t]+(?=\S)', '', texts, flags=re.MULTILINE)
                    texts = re.sub(r'\t{5,} ?', '\n', texts)
                    texts = re.sub(r'(?<=\d)\t{3,}', '\n', texts)
                    texts = texts.replace('\t Total', '\n Total')
                    texts = texts.replace('$', '')
                    texts = re.sub(r'(\(\w\),)+\*', '', texts)
                    texts = texts.replace('*', '')
                    texts = texts.replace('\t\t', '\t')
                    texts = texts.replace("\t\t\n", "\n")
                    texts = re.sub(r'(?<=\d)\t\n', '\n', texts)

                    if firts_text:
                        full_text_table += f"\n{texts}"
                    else:
                        firts_text= True
                        full_text_table = texts


                    # print(repr(f"\n----------------------------------------------------------------------------------\n{texts}"))
                    # print(f"\n----------------------------------------------------------------------------------\n{texts}")
            # print(full_text_table)
            result = StringIO(full_text_table)
            try:
                df = pd.read_csv(result, sep='\t', header=0, index_col=False)
            except:
                continue
            
            if len(df.columns) > 4:
                df.columns = ["Security", "Initial Acquisition Date", "Shares", "Value", "Percent of Net Assets"]
                df = df[(df.Security != "Security")&(df["Initial Acquisition Date"] != "Initial Acquisition Date")&(df.Shares != "Shares")&(df.Value != "Value")&(df["Percent of Net Assets"] != "Percent of Net Assets (1)")]
            else:
                df.columns = ["Security", "Initial Acquisition Date", "Shares", "Value"]
                df = df[(df.Security != "Security")&(df["Initial Acquisition Date"] != "Initial Acquisition Date")&(df.Shares != "Shares")&(df.Value != "Value")]
            
            
            df["Report Date"] = [dates[j] for i in range(len(df['Initial Acquisition Date'].values))]

            # Create a new column for sections
            df['Type'] = np.nan

            # Forward fill the section column for non-NaN 'Security' rows
            section = None
            for i, row in df.iterrows():
                try:
                    if pd.isna(row['Initial Acquisition Date']) and pd.isna(row['Shares']) and pd.isna(row['Value']) and pd.isna(row['Percent of Net Assets']):
                        section = row['Security']
                except KeyError:
                    if pd.isna(row['Initial Acquisition Date']) and pd.isna(row['Shares']) and pd.isna(row['Value']):
                        section = row['Security']
                
                df.at[i, 'Type'] = section

            try:
                # Drop the rows that are sections
                df = df.dropna(subset=['Initial Acquisition Date', 'Shares', 'Value', 'Percent of Net Assets'])
            except KeyError:
                # Drop the rows that are sections
                df = df.dropna(subset=['Initial Acquisition Date', 'Shares', 'Value'])

            # Reset the index of the DataFrame
            df = df.reset_index(drop=True)

            # Shift values to the right if 'Percent of Net Assets (1)' is empty
            # for i, row in df.iterrows():
            #     if pd.isna(row['Percent of Net Assets']):
            #         if pd.isna(row['Value']):
            #             df.at[i, 'Percent of Net Assets'] = row['Shares']
            #             df.at[i, 'Value'] = row['Initial Acquisition Date']
            #             df.at[i, 'Shares'] = np.nan
            #             df.at[i, 'Initial Acquisition Date'] = np.nan
            #             df.at[i, 'Report Date'] = np.nan
            #         else:
            #             df.at[i, 'Percent of Net Assets'] = row['Value']
            #             df.at[i, 'Value'] = row['Shares']
            #             df.at[i, 'Shares'] = row['Initial Acquisition Date']
            #             df.at[i, 'Initial Acquisition Date'] = np.nan
            #             df.at[i, 'Report Date'] = np.nan
            
            # columns = ["Type", "Security", "Initial Acquisition Date", "Report Date", "Shares", "Value", "Percent of Net Assets"]
            # df = df[columns]

            # print("\n\n")
            # with open("C:/Users/mlinares/Downloads/processed.txt", "w") as f:
            #     f.write(tabulate(df, headers='keys', tablefmt='psql'))

            # df.to_csv("test.csv", index=False)

            df['Value'] = df['Value'].str.replace(',', '')

            # Define the pattern to search for (partial match)
            pattern = re.compile(r'^Investment is issued in a private placement offering')
            original_price_text = soup.find(lambda tag: tag.name == "p" and pattern.search(tag.text)).text

            # Regular expression pattern to match numbers with commas
            pattern = r'\$\d{1,3}(?:,\d{3})*'
            # Find all matches in the text
            matches = re.findall(pattern, original_price_text)
            if matches:
                matches = matches[:-1]
                matches = [x.replace('$', '').replace(',', '') for x in matches]
            
            else:
                date = df['Report Date'][0]
                date_obt = datetime.strptime(date, "%Y-%m-%d")

                pattern = re.compile(r'^\s*As of {0}\xa0{1}, {2}, the aggregate cost'.format(date_obt.strftime("%B"), date_obt.strftime("%d"), date_obt.strftime("%Y")))
                soup = BeautifulSoup(r.text, 'html.parser')
                # paragraphs = soup.findAll('p')
                # for paragraph in paragraphs:
                #     print(repr(paragraph.text))
                original_price_text = soup.find(lambda tag: tag.name == "td" and pattern.search(tag.text))
                if original_price_text == None:
                    pattern = re.compile(r'^\n*\s*As of {0}\xa0{1}, {2}, the aggregate cost'.format(date_obt.strftime("%B"), date_obt.strftime("%d"), date_obt.strftime("%Y")))
                    driver.get(urls[4])
                    text_tag = driver.find_element(By.TAG_NAME, 'text')
                    p_tags = text_tag.find_elements(By.TAG_NAME, 'p')
                    
                    for paragraph in p_tags:
                        # print('\n\n')
                        # print(repr(str(paragraph.text)))
                        pattern = re.compile(r'^\n*\s*As of {0} {1}, {2}, the aggregate cost'.format(date_obt.strftime("%B"), date_obt.strftime("%d"), date_obt.strftime("%Y")))
                        result = pattern.search(paragraph.text)
                        if result:
                            original_price_text = paragraph.text
                            break
                else:
                    original_price_text = original_price_text.text
                # Regular expression pattern to match numbers with commas
                pattern = r'\$\d{1,3}(?:,\d{3})*'
                # Find all matches in the text
                matches = re.findall(pattern, original_price_text)
                matches = matches[:-1]
                matches = [x.replace('$', '').replace(',', '') for x in matches]



            print(original_price_text)
            print(matches)
            print(len(matches))

            df['Original Value'] = matches

            try:
                columns = ["Type", "Security", "Initial Acquisition Date", "Report Date", "Shares", "Original Value", "Value", "Percent of Net Assets"]
                df = df[columns]
            except KeyError:
                columns = ["Type", "Security", "Initial Acquisition Date", "Report Date", "Shares", "Original Value", "Value"]
                df = df[columns]
            
            df_obj = df.select_dtypes('object')

            df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

            df['Report Date'] = pd.to_datetime(df['Report Date'], format="%Y-%m-%d")
            df['Report Date'] = df['Report Date'].apply(lambda x: x.strftime('%m/%d/%Y'))
            df['Initial Acquisition Date'] = pd.to_datetime(df['Initial Acquisition Date'], format='%m/%d/%Y')
            df['Initial Acquisition Date'] = df['Initial Acquisition Date'].apply(lambda x: x.strftime('%m/%d/%Y'))

            # print("\n\n")
            # print(tabulate(df, headers='keys', tablefmt='psql'))
            date = datetime.strptime(df['Report Date'][0], '%m/%d/%Y')
            date = date.strftime('%Y_%m_%d')
            print(date)

            dataframes.append((f'{date}', df))
        
        
            
            # df.to_excel(f"Private Credit Funds.xlsx", sheet_name=f"{date}", index=False)

In [379]:
ticker = 1609211
entity = get_ticker_information(1609211)
url = f"https://www.sec.gov/edgar/search/#/category=custom&ciks=000{ticker}&entityName={entity}(CIK 000{ticker})&forms=N-CSR,N-CSRS"
# print(url)


Consolidated_Schedule_Investments(ticker, entity, url).amg_pantheon_fund()

C:\Users\mlinares\AppData\Local\Temp\ipykernel_25472\1215858170.py:196: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Co-Investments' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'Type'] = section


Investment is issued in a private placement offering and is restricted to resale. Each investment may have been purchased
on various dates and for different amounts. The date of the first purchase is reflected under Initial Acquisition Date as shown in the Consolidated Schedule of Investments. As of March 31, 2024, the aggregate cost of each investment restricted
to resale was $23,888,253, $0, $1,092,956, $2,660,401, $3,335,836, $10,829,540, $4,246,276, $11,466,453, $2,141,549, $25,049,556, $4,245,077, $5,159,391, $1,5022,158, $16,379,406, $1,295,640, $4,108,705, $14,077,320, $379,427, $212,427, $7,206,199,
$7,411,756, $1,783,591, $2,174,345, $5,120,000, $17,662,814, $18,991,370, $21,892,836, $5,072,978, $15,000,000, $636,693, $8,105,938, $14,265,651, $10,654,669, $7,799,538, $4,677,399, $7,857,364, $3,033,908, $20,389,205, $1,933,218, $35,020,279,
$4,760,163, $630,216, $2,276,020, $38,256,000, $3,376,573, $5,588,918, $8,429,116, $14,975,392, $2,231,184, $8,632,039, $29,932,910, $1,849

C:\Users\mlinares\AppData\Local\Temp\ipykernel_25472\1215858170.py:196: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Co-Investments' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'Type'] = section


Investment is issued in a private placement offering and is restricted to resale. Each investment may have been purchased
on various dates and for different amounts. The date of the first purchase is reflected under Initial Acquisition Date as shown in the Consolidated Schedule of Investments. As of September 30, 2023, the aggregate cost of each investment
restricted to resale was $23,706,092, $200,093, $1,204,726, $2,657,341, $3,335,836, $10,829,540, $3,924,009, $11,466,453, $2,141,549, $25,049,556, $4,245,077, $5,159,391, $15,022,159, $16,379,406, $1,295,640, $4,108,705, $14,061,119, $730,167,
$212,427, $7,206,199, $7,411,756, $1,783,591, $2,174,345, $5,120,000, $17,662,814, $18,953,516, $5,072,978, $15,000,000, $636,693, $8,090,514, $14,265,651, $7,799,538, $3,320,212, $7,857,364, $3,033,908, $20,384,096, $1,931,802, $34,999,929,
$4,760,163, $630,216, $2,276,020, $3,376,573, $5,588,918, $8,429,116, $14,975,392, $2,231,184, $7,337,233, $1,849,814, $3,767,531, $29,622,335, $18,819,714

C:\Users\mlinares\AppData\Local\Temp\ipykernel_25472\1215858170.py:196: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Co-Investments - 39.1%' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'Type'] = section


...

Investment is issued in a private placement offering and is restricted to resale. Each investment may have been purchased
on various dates and for different amounts. The date of the first purchase is reflected under Initial Acquisition Date as shown in the Consolidated Schedule of Investments. As of September 30, 2020, the aggregate cost of each investment
restricted to resale was $500,000, $3,927,435, $1,383,560, $4,057,409, $3,335,836, $4,018,888, $3,130,747, $1,295,640, $1,426,048, $451,888, $345,534, $1,614,019, $367,521, $2,174,345, $1,240,245, $3,378,493, $3,029,898, $2,330,057, $2,537,113,
$2,028,921, $1,524,817, $3,376,573, $2,188,776, $1,842,781, $4,065,782, $4,425,434, $1,598,735, $4,091,836, $1,157,906, $498,260, $222,654, $1,089,591, $3,887,087, $3,133,164, $3,457,191, $1,094,360, $2,318,544, $196,441, $3,012,109, $3,393,333,
$1,551, $3,198,256, $1,425,351, $1,191,349, $4,454,880, $2,799,759, $263,162, $2,007,640, $125,416, $2,173,467, $53,066, $1,001,053, $973,245, $446,56

C:\Users\mlinares\AppData\Local\Temp\ipykernel_25472\1215858170.py:196: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Co-Investments - 66.2%' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'Type'] = section


Investment is issued in a private placement offering and is restricted to resale. Each investment may have been purchased
on various dates and for different amounts. The date of the first purchase is reflected under Initial Acquisition Date as shown in the Schedule of Investments. As of March 31, 2019, the aggregate cost of each investment restricted to resale was
$500,000, $3,914,417, $561,231, $1,569,292, $4,042,827, $4,005,000, $1,295,640, $2,917,147, $365,871, $486,687, $1,608,000, $1,779,735, $2,174,345, $1,240,245, $2,330,799, $2,159,956, $2,180,760, $1,842,781, $1,590,000, $1,153,003, $497,736,
$196,370, $817,852, $3,052,699, $3,457,560, $1,108,282, $0, $424,164, $1,964,764, $316,480, $2,623,238, $3,390,965, $1,551, $3,168,777, $1,425,351, $2,360,233, $263,162, $164,041, $125,416, $570,323, $48,538, $824,859, $658,481, $437,820,
$2,670,745, $2,116,430, $3,414,446, $70,741, $143,052, $114,191, $317,359, $37,591 and $449,700, respectively, totaling $74,950,653. 
['500000', '391441

C:\Users\mlinares\AppData\Local\Temp\ipykernel_25472\1215858170.py:196: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Co-Investments - 58.6%' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'Type'] = section


Investment is issued in a private placement offering and is restricted to resale. Each investment may have been
purchased on various dates and for different amounts. The date of the first purchase is reflected under Initial Acquisition Date as shown in the Consolidated Schedule of Investments. As of September 30, 2019, the aggregate cost of each
investment restricted to resale was $500,000, $3,914,417, $1,569,873, $4,056,001, $4,017,599, $1,295,640, $2,886,893, $417,069, $342,384, $1,608,000, $367,521, $2,174,345, $1,240,245, $2,330,799, $3,391,185, $1,799,073, $3,333,333, $2,180,759,
$1,842,781, $1,590,000, $1,157,906, $497,736, $222,654, $901,787, $3,880,000, $3,094,277, $3,460,049, $1,087,789, $0, $419,454, $1,964,764, $196,441, $2,623,238, $3,393,269, $1,551, $3,168,777, $1,425,351, $1,843,522, $263,162, $558,815, $125,416,
$654,648, $49,962, $879,101, $578,783, $439,105, $764,028, $2,109,154, $3,468,698, $3,940,840, $57,076, $122,980, $116,046, $3,199,498, $299,379, $36,711 and $4